In [1]:
import re, json, pathlib, logging, time, argparse, pprint, warnings
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pds
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import polmap.clean_docs as cldc
import postprocess.postprocess as pspr
from docx2python import docx2python
import docx2python.iterators as d2piter


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [2]:
def jpbextract(jpb_docx):
     """
     Helper function that extract and stores textual elements from a JRC Project Browser document.
     The function relies on docx2python and the structure of JPB documents to extract the data.
     """
     jpbdoc_text = docx2python(jpb_docx).text

     jpbdoc_text = re.sub(r'\n{2,}', '\n\n', jpbdoc_text) #Replace repeated newlines with double newlines
     jpbdoc_text = re.sub(r'Page \d{1,2} of\n\n \d{1,2} Generated \d{1,2}.\d{1,2}.\d{2,4} \d{1,2}:\d{1,2}:\d{1,2}',
                          '', jpbdoc_text) #Remove right headers

     prj_id = re.search(r'PRJ Id\n\n(?P<prj_id>\d{5})\n\nTitle', jpbdoc_text) #Capture prj_id
     acronym = re.search(r'Acronym\n\n(?P<acronym>[\w+ -]+)\n\nContext', jpbdoc_text) #Capture Acronymm
     prj_id = prj_id.group('prj_id') if prj_id else "N/A"
     acronym = acronym.group('acronym') if acronym else "N/A"
     #Abbreviete deliverable repeats in Deliverable and Other section so that .split will occur only once for each deliverable
     jpbdoc_text = re.sub(r'Deliverable ([A-Z]\w+)', r'Del. \1', jpbdoc_text) 

     jpbdoc_text = re.sub(f'WP 2021-{prj_id}-{acronym} - Project Sheet', '', jpbdoc_text) #Remove left headers

     #Select second occurrence of  and use .start .end to find boundaries and remove it
     ith=1
     breaks = list(re.finditer('Methodologies used', jpbdoc_text))

     if len(breaks) >= ith+1:
          jpbdoc_text = f'{jpbdoc_text[:breaks[1].start()]}{jpbdoc_text[breaks[1].end():]}'
     else:
          pass

     jpbdoc_text = re.sub(r'([a-z0-9]+[ ,.;:_-]*)(\n{3,})([a-z0-9]+)', r'\1\3', jpbdoc_text) #Remove interruption in fields caused by headers

     #Define fields of interest to be captured by index matching in JBPfile.     
     jpb_fields = ('Project Portfolio', 'PRJ Id','Project Sheet','Commission priorities', 'Policy area','Methodologies used')
     jpb_dict = {}  

     for index, field_string in enumerate(jpbdoc_text.split('\n\n')):
          if field_string in jpb_fields:
               jpb_dict[field_string] = jpbdoc_text.split('\n\n')[index+1]
     
     #Split text at each Deliverable entry, remove the text before the Deliverable section, restore "Deliverable" at the beginning of each item.
     # deliverables = [f'Deliverable\n\n{deliverable}' for deliverable in jpbdoc_text.split('\n\nDeliverable\n\n')[1:]]
     #
     # re_match={'labid': r'Deliverable\n\n(?P<label>\w+)\n\n(?P<id>\(\d+\))\n\nTitle',
     #           'title': r'Title\n\n(?P<title>.*)\n\nDescription',
     #           'description': r'Description\n\n(?P<description>.*)\n\nDel\. Category',
     #           'category': r'Del\. Category\n\n(?P<category>.*)\n\nExpected release date',
     #           'type': r'Del\. Category\n\n(?P<type>.*)\n\nExpected release date',
     #           'polcyc_stage': r'Stage of Policy Cycle\n\n(?P<polcyc_stage>.*)\n\n'}
     # deliverable_fields = ('Deliverable', 'Title', 'Description', 'Del. Category', 
     #                       'Expected release date', 'Sensitive deliverable', 'Del. Type', 
     #                       'Contact', 'Del. Fun. Type', 'INS-RS', 'Note (sensitivity)', 
     #                       'Classification', 'Collaborating Unit', 'Customer DG', 'Stage of Policy Cycle')
     # fields_of_interest = ('Deliverable', 'Title', 'Description', 'Del. Category', 
     #                       'Del. Type','Sensitive deliverable', 'Classification', 'Stage of Policy Cycle')
     
     # for del_idx, deliverable in enumerate(deliverables, start=1):
     #      print(del_idx)
     #      deliverable = re.sub(r'\n{2,}', '\n\n', deliverable)
     #      items = deliverable.split('\n\n')
     #      match_dict={}

     #      for item_idx, item in enumerate(items, start=1):
     #           # try:
     #           if item in fields_of_interest:
     #                print(item)
     #                match_dict[item] = items[item_idx+1] if items[item_idx+1] not in deliverable_fields else ''
     #           # except IndexError as error:
     #           #      if item_idx == len(items):
     #           #           continue
     #           #      else:
     #           #           error
                    
               
     #      # print(deliverable)
     #      # match_dict = { key : re.match(value, deliverable).group(0) if re.match(value, deliverable) else ''
     #      #              for key, value in re_match.items()
     #      #              }
     #      # del_label_id = re.match(r'Deliverable\n\n(?P<label>\w+)\n\n(?P<id>\(\d+\))\n\nTitle', deliverable)
     #      # del_title = re.match(r'Title\n\n(?P<title>.*)\n\nDescription', deliverable)
     #      # del_desc = re.match(r'Description\n\n(?P<description>.*)\n\nDel\. Category', deliverable)
     #      # del_cat = re.match(r'Del\. Category\n\n(?P<category>.*)\n\nExpected release date', deliverable)
     #      # del_type = re.match(r'Del\. Category\n\n(?P<type>.*)\n\nExpected release date', deliverable)
     #      # del_polcyc_stage = re.match(r'Stage of Policy Cycle\n\n(?P<polcyc_stage>.*)\n\n', deliverable)
          
     #      jpb_dict[f'Deliverable {del_idx}'] = match_dict  #This
          
     
     if prj_id != jpb_dict['PRJ Id'] or prj_id == "N/A":
          warnings.warn("PRJ Id from regex not captured or not matching the Prj")

     return jpb_dict, jpbdoc_text

In [3]:
jpbdoc_text = '''"Waltz, bad nymph, for quick jigs vex." (28 letters)
"Glib jocks quiz nymph to vex dwarf." (28 letters)
"Sphinx of black quartz, judge my vow." (29 letters)
"How vexingly quick daft zebras jump!" (30 letters)
"The five boxing wizards jump quickly." (31 letters)
"Jackdaws love my big sphinx of quartz." (31 letters)
"Pack my box with five dozen liquor jugs." (32 letters)'''

ith=1
breaks = list(re.finditer('letters', jpbdoc_text))

if len(breaks) >= ith+1:
    jpbdoc_text = f'{jpbdoc_text[:breaks[1].start()]}{jpbdoc_text[breaks[1].end():]}'
else:
    pass

print(jpbdoc_text)

"Waltz, bad nymph, for quick jigs vex." (28 letters)
"Glib jocks quiz nymph to vex dwarf." (28 )
"Sphinx of black quartz, judge my vow." (29 letters)
"How vexingly quick daft zebras jump!" (30 letters)
"The five boxing wizards jump quickly." (31 letters)
"Jackdaws love my big sphinx of quartz." (31 letters)
"Pack my box with five dozen liquor jugs." (32 letters)


In [5]:
test_dict, test_text = jpbextract("input/DirD/WP_2021_PP_20172_PRJ_30440_13072021.docx")
#deliverables = [f'Deliverable\n\n{deliverable}' for deliverable in test_text.split('\n\nDeliverable\n\n')[1:]]
print(test_dict)

{'Project Sheet': 'WP 2021 PRJ 30440 GTCAP', 'Project Portfolio': '20172 - DIGI-CAP', 'PRJ Id': '30440', 'Methodologies used': 'GTCAP is currently the reference for various check methodologies e.g. orthoimagery, photo-interpretation, LPIS, area measurement and Sentinel-based monitoring under the checks by monitoring procedure.The uptake of new technologies involves further elaboration of these as well as an uptake/integration of cloud solutions, artificial intelligence and data interoperability.', 'Commission priorities': '1. The European Green Deal', 'Policy area': '1.8. Agriculture and food systems'}


In [ ]:
jpb_object = docx2python("input/DirD/WP_2021_PP_20140_PRJ_30690_13072021.docx")
jpbdoc_list = list(d2piter.iter_at_depth(jpb_object.body, 4))
# for index, item in enumerate(jpbdoc_list):
#     print(item)
#     if not (bool(jpbdoc_list[index-1]) & bool(jpbdoc_list[index+1])):
#         if jpbdoc_list[index+2] == len(jpbdoc_list):
#             break
#         print(jpbdoc_list[index])
#         del jpbdoc_list[index]
    
jpbdoc_text = jpb_object.text
jpbdoc_text = re.sub(r'\n{2,}', '\n\n', jpbdoc_text)
# for item in list(d2piter.iter_at_depth(jpb_object.body, 4)):
#     print(item)
print(jpbdoc_text)
##jpb_file.document


In [ ]:
dicts = []
for file_ in files:
    dicts.append()
#print(dicts[-1])
#dicts = [{"A": 1, "B": 2, "C": 3, "X": 4}, {"A": 5, "B": 6, "C": 7, "Y": 8}]
#df1 = pd.DataFrame(columns=('Project Portfolio', 'PRJ Id','Project Sheet','Commission priorities', 'Policy area','Methodologies used'))
df1 = pd.DataFrame(dicts)
#.from_records(dicts, columns=('Project Portfolio', 'PRJ Id','Project Sheet','Commission priorities', 'Policy area','Methodologies used')) #df1.append(dicts, ignore_index=True, sort=False)

#print(df1.head())

#with open as outpath:
df1.to_excel(f'{out_file}.xlsx')
df1.to_json(f'{out_file}.json')
print('Done!')  